In [4]:
import os
import json
import torch
import torch.nn as nn
from sklearn.utils.class_weight import compute_class_weight
import torch.optim as optim
from torch.utils.data import DataLoader, TensorDataset, ConcatDataset
from sklearn.model_selection import train_test_split
import numpy as np
from tqdm import tqdm
import random
from torch.optim.lr_scheduler import ReduceLROnPlateau

# device 정의
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

DOG_SKELETON = [
    [0, 1],  # 코에서 이마
    [0, 2],  # 코에서 입꼬리
    [2, 3],  # 입꼬리에서 아랫입술
    [1, 4],  # 이마에서 목
    [4, 5],  # 목에서 오른쪽 앞다리 시작점
    [4, 6],  # 목에서 왼쪽 앞다리 시작점
    [5, 7],  # 오른쪽 앞다리 시작점에서 오른쪽 앞발목
    [6, 8],  # 왼쪽 앞다리 시작점에서 왼쪽 앞발목
    [9, 11],  # 오른쪽 허벅지에서 오른쪽 뒷발목
    [10, 12],  # 왼쪽 허벅지에서 왼쪽 뒷발목
    [4, 13],  # 목에서 꼬리 시작점
    [13, 14],  # 꼬리 시작점에서 꼬리 끝
    [9, 13],  # 오른쪽 허벅지에서 꼬리 시작점 (오른쪽 하체)
    [10, 13], # 왼쪽 허벅지에서 꼬리 시작점 (왼쪽 하체)
    [5, 9],   # 오른쪽 앞다리 시작점에서 오른쪽 허벅지 (오른쪽 몸통)
    [6, 10],  # 왼쪽 앞다리 시작점에서 왼쪽 허벅지 (왼쪽 몸통)
    [5, 6],   # 오른쪽 앞다리 시작점에서 왼쪽 앞다리 시작점 (가슴 윤곽)
    [9, 10]   # 오른쪽 허벅지에서 왼쪽 허벅지 (엉덩이 윤곽)
]

def extract_keypoints(keypoints):
    extracted = []
    for i in range(1, 16):  # 1부터 15까지의 키포인트
        point = keypoints.get(str(i))
        if point is not None:
            extracted.extend([point['x'], point['y']])
        else:
            extracted.extend([0, 0])  # 없는 키포인트는 (0, 0)으로 처리
    return extracted

def extract_skeleton(keypoints):
    skeleton = []
    for start, end in DOG_SKELETON:
        start_point = keypoints.get(str(start+1))
        end_point = keypoints.get(str(end+1))
        if start_point and end_point:
            skeleton.extend([start_point['x'], start_point['y'], end_point['x'], end_point['y']])
        else:
            skeleton.extend([0, 0, 0, 0])
    return skeleton

def standardize_sequence_length(sequence, target_length=100):
    current_length = len(sequence)
    if current_length > target_length:
        indices = np.linspace(0, current_length-1, target_length, dtype=int)
        return [sequence[i] for i in indices]
    elif current_length < target_length:
        padding = [sequence[-1]] * (target_length - current_length)
        return sequence + padding
    else:
        return sequence

def temporal_augmentation(sequence, max_skip=2):
    augmented = []
    i = 0
    while i < len(sequence):
        augmented.append(sequence[i])
        i += random.randint(1, max_skip)
    return augmented

def reverse_sequence(sequence):
    return sequence[::-1]

def add_temporal_noise(sequence, noise_level=0.05):
    noisy_sequence = []
    for frame in sequence:
        noisy_frame = np.array(frame) + np.random.normal(0, noise_level, len(frame))
        noisy_sequence.append(noisy_frame.tolist())
    return noisy_sequence

def get_sequence_weight(sequence_length, max_length=100):
    return min(sequence_length / max_length, 1.0)

def load_json_files(folder_path):
    keypoints_data = []
    skeleton_data = []
    labels = []
    metadata = []
    total_files = 0
    processed_files = 0
    skipped_files = 0
    class_names = []
    
    for action_folder in os.listdir(folder_path):
        action_path = os.path.join(folder_path, action_folder)
        if os.path.isdir(action_path):
            class_name = action_folder.upper()
            if class_name not in class_names:
                class_names.append(class_name)
            class_index = class_names.index(class_name)
            
            for filename in os.listdir(action_path):
                if filename.endswith('.json'):
                    total_files += 1
                    file_path = os.path.join(action_path, filename)
                    with open(file_path, 'r', encoding='utf-8') as f:
                        try:
                            json_data = json.load(f)
                            keypoints_sequence = []
                            skeleton_sequence = []
                            for annotation in json_data['annotations']:
                                keypoints = extract_keypoints(annotation['keypoints'])
                                skeleton = extract_skeleton(annotation['keypoints'])
                                if keypoints and skeleton:
                                    keypoints_sequence.append(keypoints)
                                    skeleton_sequence.append(skeleton)
                            
                            if keypoints_sequence and skeleton_sequence:
                                # 시퀀스 길이 표준화
                                keypoints_sequence = standardize_sequence_length(keypoints_sequence)
                                skeleton_sequence = standardize_sequence_length(skeleton_sequence)
                                
                                # 데이터 증강 (50% 확률로 적용)
                                if random.random() < 0.5:
                                    keypoints_sequence = temporal_augmentation(keypoints_sequence)
                                    skeleton_sequence = temporal_augmentation(skeleton_sequence)
                                
                                if random.random() < 0.5:
                                    keypoints_sequence = reverse_sequence(keypoints_sequence)
                                    skeleton_sequence = reverse_sequence(skeleton_sequence)
                                
                                if random.random() < 0.5:
                                    keypoints_sequence = add_temporal_noise(keypoints_sequence)
                                    skeleton_sequence = add_temporal_noise(skeleton_sequence)
                                
                                # 시퀀스 길이에 따른 가중치 계산
                                weight = get_sequence_weight(len(keypoints_sequence))
                                
                                keypoints_data.append(keypoints_sequence)
                                skeleton_data.append(skeleton_sequence)
                                labels.append(class_index)
                                metadata.append({
                                    'pain': json_data['metadata']['owner']['pain'],
                                    'disease': json_data['metadata']['owner']['disease'],
                                    'emotion': json_data['metadata']['owner']['emotion'],
                                    'abnormal_action': json_data['metadata']['inspect']['abnormalAction'],
                                    'weight': weight  # 가중치 추가
                                })
                                processed_files += 1
                            else:
                                print(f"경고: {filename}에서 유효한 시퀀스를 추출하지 못했습니다.")
                                skipped_files += 1
                        except Exception as e:
                            print(f"오류 발생: {filename} 처리 중 {str(e)}")
                            skipped_files += 1
    
    print(f"총 파일 수: {total_files}")
    print(f"처리된 파일 수: {processed_files}")
    print(f"건너뛴 파일 수: {skipped_files}")
    print(f"클래스 목록: {class_names}")
    
    if not keypoints_data:
        raise ValueError("로드된 데이터가 없습니다. 데이터 경로와 파일을 확인하세요.")
    
    return keypoints_data, skeleton_data, labels, metadata, class_names

class ImprovedLSTMModel(nn.Module):
    def __init__(self, keypoint_size, skeleton_size, hidden_size, num_layers, num_classes):
        super(ImprovedLSTMModel, self).__init__()
        self.keypoint_lstm = nn.LSTM(keypoint_size, hidden_size, num_layers, batch_first=True, dropout=0.5)
        self.skeleton_lstm = nn.LSTM(skeleton_size, hidden_size, num_layers, batch_first=True, dropout=0.5)
        self.fc = nn.Linear(hidden_size * 2, num_classes)
        self.dropout = nn.Dropout(0.5)
    
    def forward(self, keypoints, skeleton):
        _, (h_n_keypoints, _) = self.keypoint_lstm(keypoints)
        _, (h_n_skeleton, _) = self.skeleton_lstm(skeleton)
        
        combined = torch.cat((h_n_keypoints[-1], h_n_skeleton[-1]), dim=1)
        out = self.dropout(combined)
        out = self.fc(out)
        return out

class EarlyStopping:
    def __init__(self, patience=7, verbose=False, delta=0, path='checkpoint.pt'):
        self.patience = patience
        self.verbose = verbose
        self.counter = 0
        self.best_score = None
        self.early_stop = False
        self.val_loss_min = np.Inf
        self.delta = delta
        self.path = path

    def __call__(self, val_loss, model):
        score = -val_loss

        if self.best_score is None:
            self.best_score = score
            self.save_checkpoint(val_loss, model)
        elif score < self.best_score + self.delta:
            self.counter += 1
            print(f'EarlyStopping counter: {self.counter} out of {self.patience}')
            if self.counter >= self.patience:
                self.early_stop = True
        else:
            self.best_score = score
            self.save_checkpoint(val_loss, model)
            self.counter = 0

    def save_checkpoint(self, val_loss, model):
        if self.verbose:
            print(f'Validation loss decreased ({self.val_loss_min:.6f} --> {val_loss:.6f}). Saving model ...')
        torch.save(model.state_dict(), self.path)
        self.val_loss_min = val_loss

def save_model(epoch, model, optimizer, scheduler, train_loss, val_loss, filename, all_class_names):
    torch.save({
        'epoch': epoch,
        'model_state_dict': model.state_dict(),
        'optimizer_state_dict': optimizer.state_dict(),
        'scheduler_state_dict': scheduler.state_dict(),
        'train_loss': train_loss,
        'val_loss': val_loss,
        'keypoint_size': model.keypoint_lstm.input_size,
        'skeleton_size': model.skeleton_lstm.input_size,
        'hidden_size': model.keypoint_lstm.hidden_size,
        'num_layers': model.keypoint_lstm.num_layers,
        'num_classes': model.fc.out_features,
        'all_class_names': all_class_names
    }, filename)

# 데이터 로드
train_folder = 'E:/LSTN_test/json/Training/DOG'
val_folder = 'E:/LSTN_test/json/Validation/DOG'

try:
    train_keypoints, train_skeleton, train_labels, train_metadata, train_class_names = load_json_files(train_folder)
    val_keypoints, val_skeleton, val_labels, val_metadata, val_class_names = load_json_files(val_folder)
    
    # 클래스 이름 통합
    all_class_names = list(set(train_class_names + val_class_names))
except ValueError as e:
    print(f"오류: {e}")
    exit(1)

def pad_sequences(sequences, max_length=None, padding_value=0.0):
    if max_length is None:
        max_length = max(len(seq) for seq in sequences)
    
    padded_sequences = []
    for seq in sequences:
        if len(seq) > max_length:
            padded_sequences.append(seq[:max_length])
        else:
            padding = [[padding_value] * len(seq[0])] * (max_length - len(seq))
            padded_sequences.append(seq + padding)
    return padded_sequences

keypoints_data = pad_sequences(keypoints_data)
skeleton_data = pad_sequences(skeleton_data)

# 데이터 로딩 후, 텐서로 변환하기 전에 패딩 적용
max_length = max(
    max(len(seq) for seq in train_keypoints),
    max(len(seq) for seq in train_skeleton),
    max(len(seq) for seq in val_keypoints),
    max(len(seq) for seq in val_skeleton)
)

print(f"Max sequence length: {max_length}")

# Keypoints와 Skeleton 데이터의 shape 확인
print(f"Train keypoints shape before padding: {np.array(train_keypoints).shape}")
print(f"Train skeleton shape before padding: {np.array(train_skeleton).shape}")

train_keypoints = pad_sequences(train_keypoints, max_length)
train_skeleton = pad_sequences(train_skeleton, max_length)
val_keypoints = pad_sequences(val_keypoints, max_length)
val_skeleton = pad_sequences(val_skeleton, max_length)

print(f"Train keypoints shape after padding: {train_keypoints.shape}")
print(f"Train skeleton shape after padding: {train_skeleton.shape}")

# 텐서로 변환
X_train_keypoints = torch.FloatTensor(train_keypoints)
X_train_skeleton = torch.FloatTensor(train_skeleton)
y_train = torch.LongTensor(train_labels)
X_val_keypoints = torch.FloatTensor(val_keypoints)
X_val_skeleton = torch.FloatTensor(val_skeleton)
y_val = torch.LongTensor(val_labels)

print(f"Final tensor shapes:")
print(f"X_train_keypoints: {X_train_keypoints.shape}")
print(f"X_train_skeleton: {X_train_skeleton.shape}")
print(f"y_train: {y_train.shape}")

# 데이터로더 생성
train_dataset = TensorDataset(X_train_keypoints, X_train_skeleton, y_train, torch.tensor(train_metadata))
val_dataset = TensorDataset(X_val_keypoints, X_val_skeleton, y_val, torch.tensor(val_metadata))
train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=32, shuffle=False)

# 모델 파라미터
keypoint_size = X_train_keypoints.shape[2]
skeleton_size = X_train_skeleton.shape[2]
hidden_size = 128
num_layers = 2
num_classes = len(all_class_names)

# 저장된 모델 확인 및 로드
latest_checkpoint = max([f for f in os.listdir('.') if f.startswith('improved_lstm_model_dog_epoch_')], default=None)
start_epoch = 0

if latest_checkpoint:
    print(f"최신 체크포인트 발견: {latest_checkpoint}")
    checkpoint = torch.load(latest_checkpoint, map_location=device)
    start_epoch = checkpoint['epoch'] + 1
    
    keypoint_size = checkpoint['keypoint_size']
    skeleton_size = checkpoint['skeleton_size']
    hidden_size = checkpoint['hidden_size']
    num_layers = checkpoint['num_layers']
    num_classes = checkpoint['num_classes']
    all_class_names = checkpoint['all_class_names']
    
    model = ImprovedLSTMModel(keypoint_size, skeleton_size, hidden_size, num_layers, num_classes).to(device)
    model.load_state_dict(checkpoint['model_state_dict'])
    optimizer = optim.Adam(model.parameters(), lr=0.001)
    optimizer.load_state_dict(checkpoint['optimizer_state_dict'])
    scheduler = ReduceLROnPlateau(optimizer, 'min', patience=5, factor=0.1)
    scheduler.load_state_dict(checkpoint['scheduler_state_dict'])
    
    print(f"체크포인트에서 학습 재개: 에포크 {start_epoch}")
else:
    print("새로운 학습 시작")
    model = ImprovedLSTMModel(keypoint_size, skeleton_size, hidden_size, num_layers, num_classes).to(device)
    optimizer = optim.Adam(model.parameters(), lr=0.001)
    scheduler = ReduceLROnPlateau(optimizer, 'min', patience=5, factor=0.1)

criterion = nn.CrossEntropyLoss(reduction='none')

# 학습
num_epochs = 100
patience = 20
early_stopping = EarlyStopping(patience=patience, verbose=True, path='best_model.pt')

for epoch in tqdm(range(start_epoch, num_epochs), desc="Epochs"):
    model.train()
    total_loss = 0
    correct = 0
    total = 0
    
    for batch_keypoints, batch_skeleton, batch_y, batch_meta in train_loader:
        batch_keypoints, batch_skeleton, batch_y = batch_keypoints.to(device), batch_skeleton.to(device), batch_y.to(device)
        
        optimizer.zero_grad()
        outputs = model(batch_keypoints, batch_skeleton)
        loss = criterion(outputs, batch_y)
        weights = torch.tensor([meta['weight'] for meta in batch_meta]).to(device)
        weighted_loss = (loss * weights).mean()
        weighted_loss.backward()
        optimizer.step()
        
        total_loss += weighted_loss.item()
        _, predicted = torch.max(outputs.data, 1)
        total += batch_y.size(0)
        correct += (predicted == batch_y).sum().item()
    
    train_loss = total_loss / len(train_loader)
    train_accuracy = 100 * correct / total
    
    # 검증
    model.eval()
    val_loss = 0
    val_correct = 0
    val_total = 0
    with torch.no_grad():
        for batch_keypoints, batch_skeleton, batch_y, batch_meta in val_loader:
            batch_keypoints, batch_skeleton, batch_y = batch_keypoints.to(device), batch_skeleton.to(device), batch_y.to(device)
            outputs = model(batch_keypoints, batch_skeleton)
            loss = criterion(outputs, batch_y)
            weights = torch.tensor([meta['weight'] for meta in batch_meta]).to(device)
            weighted_loss = (loss * weights).mean()
            val_loss += weighted_loss.item()
            _, predicted = torch.max(outputs.data, 1)
            val_total += batch_y.size(0)
            val_correct += (predicted == batch_y).sum().item()
    
    val_loss /= len(val_loader)
    val_accuracy = 100 * val_correct / val_total
    
    print(f'에포크 [{epoch+1}/{num_epochs}], 학습 손실: {train_loss:.4f}, 학습 정확도: {train_accuracy:.2f}%, 검증 손실: {val_loss:.4f}, 검증 정확도: {val_accuracy:.2f}%')
    
    # 10 에포크마다 모델 저장
    if (epoch + 1) % 10 == 0:
        save_model(epoch, model, optimizer, scheduler, train_loss, val_loss, f'improved_lstm_model_dog_epoch_{epoch+1}.pt', all_class_names)
    
    # Early Stopping 체크
    early_stopping(val_loss, model)
    if early_stopping.early_stop:
        print("Early stopping")
        break
    
    scheduler.step(val_loss)

print("학습 완료")

# 최종 모델 저장
save_model(epoch, model, optimizer, scheduler, train_loss, val_loss, 'improved_lstm_model_dog_final.pt', all_class_names)

# 최상의 모델 로드
model.load_state_dict(torch.load('best_model.pt'))

# 전체 데이터셋에 대한 평가
full_dataset = ConcatDataset([train_dataset, val_dataset])
full_loader = DataLoader(full_dataset, batch_size=32, shuffle=False)
all_meta = train_metadata + val_metadata

model.eval()
correct = 0
total = 0
all_predictions = []
all_true_labels = []

with torch.no_grad():
    for batch_keypoints, batch_skeleton, batch_y, _ in full_loader:
        batch_keypoints, batch_skeleton, batch_y = batch_keypoints.to(device), batch_skeleton.to(device), batch_y.to(device)
        outputs = model(batch_keypoints, batch_skeleton)
        _, predicted = torch.max(outputs.data, 1)
        total += batch_y.size(0)
        correct += (predicted == batch_y).sum().item()
        all_predictions.extend(predicted.cpu().numpy())
        all_true_labels.extend(batch_y.cpu().numpy())

print(f'전체 데이터셋 정확도: {100 * correct / total:.2f}%')

# 예측 결과와 메타데이터 함께 표시
for i, (pred, true) in enumerate(zip(all_predictions, all_true_labels)):
    pred_class = all_class_names[pred]
    true_class = all_class_names[true]
    print(f"샘플 {i+1}:")
    print(f"  예측: {pred_class}, 실제: {true_class}")
    print(f"  메타데이터:")
    print(f"    통증: {all_meta[i]['pain']}")
    print(f"    질병: {all_meta[i]['disease']}")
    print(f"    감정: {all_meta[i]['emotion']}")
    print(f"    비정상 행동: {all_meta[i]['abnormal_action']}")
    print()

print("최종 모델이 'improved_lstm_model_dog_final.pt'로 저장되었습니다.")

KeyboardInterrupt: 

In [2]:
%pip install numpy

Note: you may need to restart the kernel to use updated packages.


DEPRECATION: Loading egg at c:\users\administrator\appdata\local\programs\python\python312\lib\site-packages\torchlight-1.0-py3.12.egg is deprecated. pip 24.3 will enforce this behaviour change. A possible replacement is to use pip for package installation. Discussion can be found at https://github.com/pypa/pip/issues/12330


In [2]:
import os
import json
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, TensorDataset, ConcatDataset
from sklearn.model_selection import train_test_split
from sklearn.utils.class_weight import compute_class_weight
import numpy as np
from tqdm import tqdm
import random
from torch.optim.lr_scheduler import ReduceLROnPlateau
from sklearn.metrics import f1_score, precision_score, recall_score

# GPU 사용 가능 여부 확인
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

# 강아지 골격 정의
DOG_SKELETON = [
    [0, 1], [0, 2], [2, 3], [1, 4], [4, 5], [4, 6], [5, 7], [6, 8],
    [9, 11], [10, 12], [4, 13], [13, 14], [9, 13], [10, 13], [5, 9],
    [6, 10], [5, 6], [9, 10]
]

def extract_keypoints(keypoints):
    extracted = []
    for i in range(1, 16):
        point = keypoints.get(str(i))
        if point is not None:
            extracted.extend([point['x'], point['y']])
        else:
            extracted.extend([float('nan'), float('nan')])  # 누락된 키포인트를 NaN으로 처리
    return extracted

def extract_skeleton(keypoints):
    skeleton = []
    for start, end in DOG_SKELETON:
        start_point = keypoints.get(str(start+1))
        end_point = keypoints.get(str(end+1))
        if start_point and end_point:
            skeleton.extend([start_point['x'], start_point['y'], end_point['x'], end_point['y']])
        else:
            skeleton.extend([float('nan'), float('nan'), float('nan'), float('nan')])  # 누락된 골격 정보를 NaN으로 처리
    return skeleton

def standardize_sequence_length(sequence, target_length=100):
    current_length = len(sequence)
    if current_length > target_length:
        indices = np.linspace(0, current_length-1, target_length, dtype=int)
        return [sequence[i] for i in indices]
    elif current_length < target_length:
        padding = [sequence[-1]] * (target_length - current_length)
        return sequence + padding
    else:
        return sequence

def temporal_augmentation(sequence, max_skip=2):
    augmented = []
    i = 0
    while i < len(sequence):
        augmented.append(sequence[i])
        i += random.randint(1, max_skip)
    return augmented

def reverse_sequence(sequence):
    return sequence[::-1]

def add_temporal_noise(sequence, noise_level=0.05):
    noisy_sequence = []
    for frame in sequence:
        noisy_frame = np.array(frame) + np.random.normal(0, noise_level, len(frame))
        noisy_sequence.append(noisy_frame.tolist())
    return noisy_sequence

def get_sequence_weight(sequence_length, max_length=100):
    return min(sequence_length / max_length, 1.0)

def pad_sequences(sequences, max_length=None, padding_value=0.0):
    if max_length is None:
        max_length = max(len(seq) for seq in sequences)
    
    padded_sequences = []
    for seq in sequences:
        if len(seq) > max_length:
            padded_sequences.append(seq[:max_length])
        else:
            padding = [[padding_value] * len(seq[0])] * (max_length - len(seq))
            padded_sequences.append(seq + padding)
    return padded_sequences

def load_json_files(folder_path):
    keypoints_data = []
    skeleton_data = []
    labels = []
    metadata = []
    total_files = 0
    processed_files = 0
    skipped_files = 0
    class_names = []
    
    for action_folder in os.listdir(folder_path):
        action_path = os.path.join(folder_path, action_folder)
        if os.path.isdir(action_path):
            class_name = action_folder.upper()
            if class_name not in class_names:
                class_names.append(class_name)
            class_index = class_names.index(class_name)
            
            for filename in os.listdir(action_path):
                if filename.endswith('.json'):
                    total_files += 1
                    file_path = os.path.join(action_path, filename)
                    try:
                        with open(file_path, 'r', encoding='utf-8') as f:
                            json_data = json.load(f)
                            keypoints_sequence = []
                            skeleton_sequence = []
                            for annotation in json_data['annotations']:
                                keypoints = extract_keypoints(annotation['keypoints'])
                                skeleton = extract_skeleton(annotation['keypoints'])
                                if keypoints and skeleton:
                                    keypoints_sequence.append(keypoints)
                                    skeleton_sequence.append(skeleton)
                            
                            if keypoints_sequence and skeleton_sequence:
                                # 시퀀스 길이 표준화
                                keypoints_sequence = standardize_sequence_length(keypoints_sequence)
                                skeleton_sequence = standardize_sequence_length(skeleton_sequence)
                                
                                # 데이터 증강 (50% 확률로 적용)
                                if random.random() < 0.5:
                                    keypoints_sequence = temporal_augmentation(keypoints_sequence)
                                    skeleton_sequence = temporal_augmentation(skeleton_sequence)
                                
                                if random.random() < 0.5:
                                    keypoints_sequence = reverse_sequence(keypoints_sequence)
                                    skeleton_sequence = reverse_sequence(skeleton_sequence)
                                
                                if random.random() < 0.5:
                                    keypoints_sequence = add_temporal_noise(keypoints_sequence)
                                    skeleton_sequence = add_temporal_noise(skeleton_sequence)
                                
                                # 시퀀스 길이에 따른 가중치 계산
                                weight = get_sequence_weight(len(keypoints_sequence))
                                
                                keypoints_data.append(keypoints_sequence)
                                skeleton_data.append(skeleton_sequence)
                                labels.append(class_index)
                                metadata.append({
                                    'pain': json_data['metadata']['owner']['pain'],
                                    'disease': json_data['metadata']['owner']['disease'],
                                    'emotion': json_data['metadata']['owner']['emotion'],
                                    'abnormal_action': json_data['metadata']['inspect']['abnormalAction'],
                                    'weight': weight
                                })
                                processed_files += 1
                            else:
                                print(f"경고: {filename}에서 유효한 시퀀스를 추출하지 못했습니다.")
                                skipped_files += 1
                    except json.JSONDecodeError as e:
                        print(f"JSON 디코딩 오류: {filename} - {str(e)}")
                        skipped_files += 1
                    except KeyError as e:
                        print(f"키 오류: {filename} - {str(e)}")
                        skipped_files += 1
                    except Exception as e:
                        print(f"예상치 못한 오류: {filename} - {str(e)}")
                        skipped_files += 1
    
    print(f"총 파일 수: {total_files}")
    print(f"처리된 파일 수: {processed_files}")
    print(f"건너뛴 파일 수: {skipped_files}")
    print(f"클래스 목록: {class_names}")
    
    if not keypoints_data:
        raise ValueError("로드된 데이터가 없습니다. 데이터 경로와 파일을 확인하세요.")
    
    # 패딩 적용
    keypoints_data = pad_sequences(keypoints_data)
    skeleton_data = pad_sequences(skeleton_data)
    
    return keypoints_data, skeleton_data, labels, metadata, class_names

class ImprovedLSTMModel(nn.Module):
    def __init__(self, keypoint_size, skeleton_size, hidden_size, num_layers, num_classes, dropout_rate=0.5):
        super(ImprovedLSTMModel, self).__init__()
        self.keypoint_lstm = nn.LSTM(keypoint_size, hidden_size, num_layers, batch_first=True, dropout=dropout_rate)
        self.skeleton_lstm = nn.LSTM(skeleton_size, hidden_size, num_layers, batch_first=True, dropout=dropout_rate)
        self.attention = nn.MultiheadAttention(hidden_size * 2, num_heads=4)
        self.fc = nn.Linear(hidden_size * 2, num_classes)
        self.dropout = nn.Dropout(dropout_rate)
    
    def forward(self, keypoints, skeleton):
        keypoint_out, _ = self.keypoint_lstm(keypoints)
        skeleton_out, _ = self.skeleton_lstm(skeleton)
        
        combined = torch.cat((keypoint_out, skeleton_out), dim=2)
        
        # Apply attention mechanism
        attn_output, _ = self.attention(combined, combined, combined)
        
        # Global average pooling
        pooled = torch.mean(attn_output, dim=1)
        
        out = self.dropout(pooled)
        out = self.fc(out)
        return out

class EarlyStopping:
    def __init__(self, patience=7, verbose=False, delta=0, path='checkpoint.pt'):
        self.patience = patience
        self.verbose = verbose
        self.counter = 0
        self.best_score = None
        self.early_stop = False
        self.val_loss_min = np.Inf
        self.delta = delta
        self.path = path

    def __call__(self, val_loss, model):
        score = -val_loss

        if self.best_score is None:
            self.best_score = score
            self.save_checkpoint(val_loss, model)
        elif score < self.best_score + self.delta:
            self.counter += 1
            print(f'EarlyStopping counter: {self.counter} out of {self.patience}')
            if self.counter >= self.patience:
                self.early_stop = True
        else:
            self.best_score = score
            self.save_checkpoint(val_loss, model)
            self.counter = 0

    def save_checkpoint(self, val_loss, model):
        if self.verbose:
            print(f'Validation loss decreased ({self.val_loss_min:.6f} --> {val_loss:.6f}). Saving model ...')
        torch.save(model.state_dict(), self.path)
        self.val_loss_min = val_loss

def save_model(epoch, model, optimizer, scheduler, train_loss, val_loss, filename, all_class_names):
    torch.save({
        'epoch': epoch,
        'model_state_dict': model.state_dict(),
        'optimizer_state_dict': optimizer.state_dict(),
        'scheduler_state_dict': scheduler.state_dict(),
        'train_loss': train_loss,
        'val_loss': val_loss,
        'keypoint_size': model.keypoint_lstm.input_size,
        'skeleton_size': model.skeleton_lstm.input_size,
        'hidden_size': model.keypoint_lstm.hidden_size,
        'num_layers': model.keypoint_lstm.num_layers,
        'num_classes': model.fc.out_features,
        'all_class_names': all_class_names
    }, filename)

# 데이터 로드
train_folder = 'E:/LSTN_test/json/Training/DOG'
val_folder = 'E:/LSTN_test/json/Validation/DOG'

try:
    train_keypoints, train_skeleton, train_labels, train_metadata, train_class_names = load_json_files(train_folder)
    val_keypoints, val_skeleton, val_labels, val_metadata, val_class_names = load_json_files(val_folder)
    
    # 클래스 이름 통합
    all_class_names = list(set(train_class_names + val_class_names))
except ValueError as e:
    print(f"오류: {e}")
    exit(1)

# 클래스 가중치 계산
train_labels_np = np.array(train_labels)
class_weights = compute_class_weight(class_weight='balanced', classes=np.unique(train_labels_np), y=train_labels_np)
class_weights = torch.FloatTensor(class_weights).to(device)

print("train_keypoints shape:", np.array(train_keypoints).shape)
print("train_skeleton shape:", np.array(train_skeleton).shape)

# 텐서로 변환
X_train_keypoints = torch.FloatTensor(train_keypoints)
X_train_skeleton = torch.FloatTensor(train_skeleton)
y_train = torch.LongTensor(train_labels)
X_val_keypoints = torch.FloatTensor(val_keypoints)
X_val_skeleton = torch.FloatTensor(val_skeleton)
y_val = torch.LongTensor(val_labels)

# 데이터로더 생성
train_dataset = TensorDataset(X_train_keypoints, X_train_skeleton, y_train)
val_dataset = TensorDataset(X_val_keypoints, X_val_skeleton, y_val)
train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=32, shuffle=False)

# 모델 파라미터
keypoint_size = X_train_keypoints.shape[2]
skeleton_size = X_train_skeleton.shape[2]
hidden_size = 128
num_layers = 2
num_classes = len(all_class_names)

# 저장된 모델 확인 및 로드
latest_checkpoint = max([f for f in os.listdir('.') if f.startswith('improved_lstm_model_dog_epoch_')], default=None)
start_epoch = 0

if latest_checkpoint:
    print(f"최신 체크포인트 발견: {latest_checkpoint}")
    checkpoint = torch.load(latest_checkpoint, map_location=device)
    start_epoch = checkpoint['epoch'] + 1
    
    keypoint_size = checkpoint['keypoint_size']
    skeleton_size = checkpoint['skeleton_size']
    hidden_size = checkpoint['hidden_size']
    num_layers = checkpoint['num_layers']
    num_classes = checkpoint['num_classes']
    all_class_names = checkpoint['all_class_names']
    
    model = ImprovedLSTMModel(keypoint_size, skeleton_size, hidden_size, num_layers, num_classes).to(device)
    model.load_state_dict(checkpoint['model_state_dict'])
    optimizer = optim.Adam(model.parameters())
    optimizer.load_state_dict(checkpoint['optimizer_state_dict'])
    scheduler = ReduceLROnPlateau(optimizer, mode='min', factor=0.5, patience=5, verbose=True)
    scheduler.load_state_dict(checkpoint['scheduler_state_dict'])
    print(f"모델을 에포크 {start_epoch}부터 계속 훈련합니다.")
else:
    print("새 모델로 훈련을 시작합니다.")
    model = ImprovedLSTMModel(keypoint_size, skeleton_size, hidden_size, num_layers, num_classes).to(device)
    optimizer = optim.Adam(model.parameters())
    scheduler = ReduceLROnPlateau(optimizer, mode='min', factor=0.5, patience=5, verbose=True)

# 손실 함수 정의
criterion = nn.CrossEntropyLoss(weight=class_weights)

# 조기 종료 설정
patience = 15  # 원하는 patience 값으로 설정
early_stopping = EarlyStopping(patience=10, verbose=True)

# 훈련 에포크 수 정의
num_epochs = 100

# 훈련 루프
for epoch in range(start_epoch, num_epochs):
    model.train()
    train_loss = 0.0
    train_correct = 0
    train_total = 0
    
    for batch_keypoints, batch_skeleton, batch_labels in tqdm(train_loader, desc=f"Epoch {epoch+1}/{num_epochs}"):
        batch_keypoints, batch_skeleton, batch_labels = batch_keypoints.to(device), batch_skeleton.to(device), batch_labels.to(device)
        
        optimizer.zero_grad()
        outputs = model(batch_keypoints, batch_skeleton)
        loss = criterion(outputs, batch_labels)
        loss.backward()
        optimizer.step()
        
        train_loss += loss.item()
        _, predicted = outputs.max(1)
        train_total += batch_labels.size(0)
        train_correct += predicted.eq(batch_labels).sum().item()
    
    train_loss /= len(train_loader)
    train_accuracy = train_correct / train_total
    
    # 검증
    model.eval()
    val_loss = 0.0
    val_correct = 0
    val_total = 0
    all_predictions = []
    all_labels = []
    
    with torch.no_grad():
        for batch_keypoints, batch_skeleton, batch_labels in val_loader:
            batch_keypoints, batch_skeleton, batch_labels = batch_keypoints.to(device), batch_skeleton.to(device), batch_labels.to(device)
            
            outputs = model(batch_keypoints, batch_skeleton)
            loss = criterion(outputs, batch_labels)
            
            val_loss += loss.item()
            _, predicted = outputs.max(1)
            val_total += batch_labels.size(0)
            val_correct += predicted.eq(batch_labels).sum().item()
            
            all_predictions.extend(predicted.cpu().numpy())
            all_labels.extend(batch_labels.cpu().numpy())
    
    val_loss /= len(val_loader)
    val_accuracy = val_correct / val_total
    
    # F1 점수, 정밀도, 재현율 계산
    f1 = f1_score(all_labels, all_predictions, average='weighted')
    precision = precision_score(all_labels, all_predictions, average='weighted')
    recall = recall_score(all_labels, all_predictions, average='weighted')
    
    print(f"Epoch {epoch+1}/{num_epochs}")
    print(f"Train Loss: {train_loss:.4f}, Train Accuracy: {train_accuracy:.4f}")
    print(f"Val Loss: {val_loss:.4f}, Val Accuracy: {val_accuracy:.4f}")
    print(f"F1 Score: {f1:.4f}, Precision: {precision:.4f}, Recall: {recall:.4f}")
    
    # 학습률 조정
    scheduler.step(val_loss)
    
    # 모델 저장
    if val_loss < best_val_loss:
        best_val_loss = val_loss
        save_model(epoch, model, optimizer, scheduler, train_loss, val_loss, f'improved_lstm_model_dog_epoch_{epoch+1}.pth', all_class_names)
    
    # 조기 종료 확인
    early_stopping(val_loss, model)
    if early_stopping.early_stop:
        print("조기 종료")
        break

print("훈련 완료")

총 파일 수: 39537
처리된 파일 수: 39537
건너뛴 파일 수: 0
클래스 목록: ['BODYLOWER', 'BODYSCRATCH', 'BODYSHAKE', 'FEETUP', 'FOOTUP', 'HEADING', 'LYING', 'MOUNTING', 'SIT', 'TAILING', 'TAILLOW', 'TURN', 'WALKRUN']
총 파일 수: 4949
처리된 파일 수: 4949
건너뛴 파일 수: 0
클래스 목록: ['BODYLOWER', 'BODYSCRATCH', 'BODYSHAKE', 'FEETUP', 'FOOTUP', 'HEADING', 'LYING', 'MOUNTING', 'SIT', 'TAILING', 'TAILLOW', 'TURN', 'WALKRUN']
train_keypoints shape: (39537, 100, 30)
train_skeleton shape: (39537, 100, 72)
새 모델로 훈련을 시작합니다.


c:\Users\Administrator\AppData\Local\Programs\Python\Python312\Lib\site-packages\torch\optim\lr_scheduler.py:60: UserWarning: The verbose parameter is deprecated. Please use get_last_lr() to access the learning rate.
  warnings.warn(


NameError: name 'num_epochs' is not defined